In [1]:
import sys

sys.path.append('../')

In [2]:
import quantstats as qs

In [3]:
stock = qs.utils.download_returns('META')

In [ ]:
qs.reports.full(stock, "SPY")

In [4]:
qs.reports.html(stock, "SPY", text_description="""
    <p>Demonstration of a real portfolio comprising simple strategies.
    No machine learning algorithms are employed in either the strategies or the portfolio.
    No forecasting is conducted to optimize siding or sizing tasks.
    The performance of the strategies is not monitored. Consequently, there may be strategies
    affecting overall performance.</p>
    <p><strong>Important</strong>: Transaction costs are not taken into account.</p>""")

                    SPY Strategy
Beta                  -     1.24
Alpha                 -     0.13
Correlation           -   52.42%
                                
Risk-Free Rate     0.0%     0.0%
...                 ...      ...
Avg. Down Month  -4.61%   -8.75%
Win Days         54.78%   52.46%
Win Month        67.63%   61.87%
Win Quarter      78.72%   63.83%
Win Year          75.0%    75.0%

[75 rows x 2 columns]


In [4]:
import pandas as pd

In [5]:
returns = stock
benchmark="SPY"
rf=0.0
grayscale=False
title="Strategy Tearsheet"
output=None
compounded=True
periods_per_year=252
download_filename="tearsheet.html"
figfmt="svg"
template_path=None
match_dates=True

In [6]:
if match_dates:
  returns = returns.dropna()

win_year, win_half_year = qs.reports._get_trading_periods(periods_per_year)

tpl = ""
with open("../quantstats/report.html") as f:
    tpl = f.read()
    f.close()

# prepare timeseries
if match_dates:
    returns = returns.dropna()
returns = qs.utils._prepare_returns(returns)

strategy_title = "Strategy"
if isinstance(returns, pd.DataFrame):
    if len(returns.columns) > 1 and isinstance(strategy_title, str):
        strategy_title = list(returns.columns)

In [7]:
if benchmark is not None:
    benchmark_title = "Benchmark"
    if benchmark_title is None:
        if isinstance(benchmark, str):
            benchmark_title = benchmark
        elif isinstance(benchmark, pd.Series):
            benchmark_title = benchmark.name
        elif isinstance(benchmark, pd.DataFrame):
            benchmark_title = benchmark[benchmark.columns[0]].name

    tpl = tpl.replace(
        "{{benchmark_title}}", f"Benchmark is {benchmark_title.upper()} | "
    )
    benchmark = qs.utils._prepare_benchmark(benchmark, returns.index, rf)
    if match_dates is True:
        returns, benchmark = qs.reports._match_dates(returns, benchmark)
else:
    benchmark_title = None

date_range = returns.index.strftime("%e %b, %Y")
tpl = tpl.replace("{{date_range}}", date_range[0] + " - " + date_range[-1])
tpl = tpl.replace("{{title}}", title)
tpl = tpl.replace("{{v}}", "0.0.63")

if benchmark is not None:
    benchmark.name = benchmark_title
if isinstance(returns, pd.Series):
    returns.name = strategy_title
elif isinstance(returns, pd.DataFrame):
    returns.columns = strategy_title

In [8]:
mtrx = qs.reports.metrics(
    returns=returns,
    benchmark=benchmark,
    rf=rf,
    display=False,
    mode="full",
    sep=True,
    internal="True",
    compounded=compounded,
    periods_per_year=periods_per_year,
    prepare_returns=False,
    benchmark_title=benchmark_title,
    strategy_title=strategy_title,
)[2:]

In [9]:
mtrx.iloc[:6]

,Benchmark,Strategy
Beta,-,1.24
Alpha,-,0.13
Correlation,-,52.42%
Treynor Ratio,-,730.12%
,,
Risk-Free Rate,0.0%,0.0%
